## IS590PR - Fall 2017
## Assignment 5:  Analysis with multi-file or multi-table data using Pandas
### Data Set: Federal Elections Commission

http://classic.fec.gov/finance/disclosure/ftpdet.shtml files for download and the metadata.

Refer to examples in the Notebook I used in class as needed for starting points.  It is here: https://github.com/iSchool-590PR-2017Fall/590PR-examples/blob/master/FEC_pandas.ipynb

Also refer to Chapters 5 & 8 of the McKinney book for more examples. 

Note you may have to add support for the other data files in the FEC data set. Read their metadata pages to understand their contents.

# Instructions:
* **DO NOT copy the whole lecture notebook** into this one, because a lot of it does not pertain directly to the analysis in the assignment.
* **Choose 3 of the analytical queries you are going to implement from the lists below. You must try at least 1 from the multi-file queries. Then, for your fourth query, come up with something else different from any of these to implement. Make sure your English description of its purpose is clear.**
* Add the code necessary to complete each one using Pandas DataFrames and/or Pandas Series. Clearly label each one with "Markdown" cells above them, and reasonable amounts of documentation between any intermediate results to show how you created the result. 
* Once you have things working, delete any wrong or experimental code that you may still have lingering, so that what remains shows just the complete start-to-finish process of loading the data and producing your analytical results.
* Save the notebook with your output shown.
* Commit & Push this notebook file before the due date so I can see your work in GitHub Classroom for grading & feedback.

# QUERIES TO CHOOSE FROM:

[General tips: Unless stated otherwise, you may use just a single 2-year data set to compute any of these. Any time a query description has 'per' or 'each', that means you need to use groupby on the column(s) that follow. But some queries described without those words also still need groupby]

## Single-file queries:
* List all individuals who donated a TOTAL of over 100,000 dollars. Show name, city, state, employer, occupation, and total amount. [Tip: You must groupby name+city+state or by name+zipcode]
* List top 10 cities by total individual donations. Try creating a visual barchart with matplotlib.
* Compute the total amount of donations received PER MONTH from all individuals together, listing in date order. [Tips: You'll have to correct the import parsing of TRANSACTION_DT in the itcont.txt file -- The original text files have dates like '13102017' for Oct. 13, 2017. ]
* Total Expenditures by all committees per month. [Tip: You'll have to correct the import parsing of TRANSACTION_DT in the oppexp.txt file -- The original text files have dates like '13/10/2017' for Oct. 13, 2017.]
* List name, city, and state of individual people who made more than 1,000 donation **transactions**.  It's crazy, but there's even someone here in Champaign if you're using the 2015-2016 data.
* List the top 20 professions (Occupations) for total donations.

* For donations directly to the presidential election only (2015-2016 files), determine total individual donations PER STATE.


## Multi-file queries (needing merge or join).  Some of these are good ways to see corruption in action:
* Find people who donated over 5000 directly to a Senate or House candidate who's not even in their own state, and show the details. [Tip: You'll need to compare the candidate's state to the individual donor's state after you get them joined.]
* List every INCUMBENT Senate & House candidate alphabetically, showing their Name, State, and Party affiliation. For each one, list the name of every Corporation or other PAC that has "bought" them with huge donations (for example, totaling more than 25,000 dollars each). [Incumbent means they're already in office and are running for re-election. This is viewable in the "CAND_ICI" column.]
* Join Committees with the "itpas" file and the Candidate file. Then use groupby properly to build a dataframe with CMTE_ID as the index, a column for Committee name, and columns for each political party of interest. In the party columns, it shows how much total money it spent on any candidates from each party.  With this we can see things like how strongly the National Rifle Association and "Big Oil" companies support Republicans and how strongly many labor unions support Democrats, or how organizations protecting natural resources support Democrats, Green Party, or other fringe parties, etc. [The list of parties coded in the database is shown here: http://classic.fec.gov/finance/disclosure/metadata/DataDictionaryPartyCodeDescriptions.shtml]

* Download all 3 individual donations files (itcont.txt) from 2014, 2016, and 2018. Figure out how to concatenate them (just the columns you need) into a single DataFrame covering that whole range from 2013-2018. Then use that to compute an analysis of questions such as:
    * Political activity is much higher for Presidential elections than in other years. Find out whether the level of donations to Senate and House campaigns goes up or down at that time compared to other years.
    * Find out how many and what percentage of individual donors have contributed to BOTH a Democratic and a Republican candidate at some point.
    

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib

####  First create functions for processing data into a format so that the data values and its header values could all be loaded into a pandas dataframe

In [2]:
def get_header(header_file) -> list:
    """
    Opens the passed in discription file (They would be in csv format containing only one line of column names)
    Read the line of the file, and use strip() and split(',') to get rid of unwanted spaces and returning a list of 
    column names that are seperated by the comma.
    
    :param header_file: CSV files with one line of data that would be served as the column names (header row)
    :return columns: A list of strings containing the column names
    """
    with open(header_file, 'r') as cols:
        columns = cols.readline().strip().split(',')
    return columns

#### Generalize the creation of pandas dataframe on the data file and its header name list for easier calling to analyze different files and data

In [3]:
def CSV_DF(data_file, header_file, sep = "|", index_col = False):
    """
    Generalize the creation of the pandas dataframe from the csv data and header files for easier calling and reusability
    """
    header_list = get_header(header_file)
    return pd.read_csv(data_file, sep = sep, index_col = index_col, names = header_list, encoding = 'utf-8')


In [4]:
# Total Expenditures by all committees per month. [Tip: You'll have to correct the import parsing of TRANSACTION_DT in the oppexp.txt file -- The original text files have dates like '13/10/2017' for Oct. 13, 2017.]

In [5]:
# operating_Expenditures = "./operating_Expenditures/"

In [6]:
# opExp_df = CSV_DF(operating_Expenditures + "oppexp.txt", operating_Expenditures + "oppexp_header_file.csv")

In [7]:
# Function to see if their are date values we'll have to correct the import parsing of TRANSACTION_DT in the oppexp.txt file 
# whether the original text files have dates like '13/10/2017' for Oct. 13, 2017.
# So we look at all the dates to see if there are any dates that have the first two digits that represents months larger than 12
# if there are, it means that incorrect expression of dates exists and we will need to change it accordingly
def find_abnormal(data):
    try:
        if int(data[0:2]) > 12:
            print(data)
    except:
        print(data)

In [8]:
# As we can see from the output, no date's expression starts with days as none of the first two digits are larger than 12
# The only problem is that there are missing values of dates, so we won't need to change the values of the dates as they are all in correct format
# opExp_df.TRANSACTION_DT.apply(find_abnormal)

In [9]:
# experiment2017 = "./operating_Expenditures/2017/oppexp.txt"

In [10]:
# experiment = CSV_DF(experiment2017, "./operating_Expenditures/2017/" + "oppexp_header_file.csv")

In [11]:
# def find_abnormal(data):
#     try:
#         if int(data[0:2]) >= 13:
#             print(data)
#     except:
#         print(data)

In [12]:
# experiment.TRANSACTION_DT.apply(find_abnormal)

In [13]:
# opExp_df.columns

In [14]:
# group_df = opExp_df.groupby(["TRANSACTION_DT"]).head()

In [15]:
# group_df.size

In [16]:
# opExp_df.size

In [17]:
# op_month_df = opExp_df.loc[:,['CMTE_ID',"TRANSACTION_DT","TRANSACTION_AMT"]]

In [18]:
# op_month_df.groupby(['CMTE_ID',"TRANSACTION_DT"])

In [19]:
def find_month(dates):
    
    return dates[0:2]

In [20]:
# op_month_df["Transaction_month"] = op_month_df.TRANSACTION_DT.dropna().apply(find_month)

In [21]:
# op_permonth_df = op_month_df.groupby(['CMTE_ID',"Transaction_month"])

In [5]:
# op_permonth_df.TRANSACTION_AMT.sum()

In [6]:
# opExp_df.TRANSACTION_AMT